In [109]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [110]:
import json
import roma
import torch
import numpy as np
from scipy.spatial.transform import Rotation 

FILE_1 = "/dust3r/masked_dust3r/data/jackal_training_data_0/transforms_1.json"
FILE_2 = "/dust3r/masked_dust3r/data/jackal_training_data_0/transforms_2.json"

with open(FILE_1, "r") as f:
    data_1 = json.load(f)
with open(FILE_2, "r") as f:
    data_2 = json.load(f)

In [111]:
file_path_1 = [f["file_path"] for f in data_1["frames"]]
file_path_2 = [f["file_path"] for f in data_2["frames"]]

# Find matching keys in data_1 and data_2
matching_keys = []
for i, f_1 in enumerate(file_path_1):
    for j, f_2 in enumerate(file_path_2):
        if f_1 == f_2:
            matching_keys.append((i, j))
            break
print(matching_keys)

X = []
Y = []

for (i,j) in matching_keys:
    X.append(data_1["frames"][i]["transform_matrix"])
    Y.append(data_2["frames"][j]["transform_matrix"])

X = torch.tensor(X)
Y = torch.tensor(Y)

X = X[:,:3,3]
Y = Y[:,:3,3]

X = X.unsqueeze(0)
Y = Y.unsqueeze(0)

(R,t,s) = roma.rigid_points_registration(Y,X,compute_scaling=True)

corr = torch.tensor(np.zeros((4,4)))
corr[:3,:3] = R[0]
corr[:3,3] = t[0]/s[0]
corr[3,3] = 1
corr = corr.numpy()

print(corr)

[(20, 0), (21, 1), (22, 2), (23, 3), (24, 4), (25, 5), (26, 6), (27, 7), (28, 8), (29, 9)]
[[-0.35816464  0.90679932  0.22233546  0.10347023]
 [-0.90166324 -0.39774653  0.16970874 -0.12124808]
 [ 0.24232499 -0.13968807  0.96008641  0.00747052]
 [ 0.          0.          0.          1.        ]]


In [114]:
all_frames = {}

for f in data_1["frames"]:
    all_frames[f["file_path"]] = f

for f in data_2["frames"]:
    transform_f = f.copy()
    transform_f["transform_matrix"] = np.dot(corr, transform_f["transform_matrix"]).tolist()
    all_frames[f["file_path"]] = transform_f

print(all_frames)

{'masked_images/0.png': {'file_path': 'masked_images/0.png', 'transform_matrix': [[0.5832669138908386, -0.6942539215087891, 0.42167627811431885, 0.15659742057323456], [-0.7428847551345825, -0.6658796072006226, -0.06874813884496689, -0.11959019303321838], [0.32851433753967285, -0.27315840125083923, -0.9041363000869751, 0.028364472091197968], [0.0, 0.0, 0.0, 1.0]], 'mask_path': 'masks/0.png', 'fl_x': 1204.841537475586, 'fl_y': 1204.841537475586, 'cx': 640.0, 'cy': 360.0}, 'masked_images/1.png': {'file_path': 'masked_images/1.png', 'transform_matrix': [[0.5058984756469727, -0.7568375468254089, 0.4138403534889221, 0.1549302637577057], [-0.7972010970115662, -0.5934606194496155, -0.11079257726669312, -0.13495416939258575], [0.3294498920440674, -0.27386417984962463, -0.9035824537277222, 0.02838529460132122], [0.0, 0.0, 0.0, 1.0]], 'mask_path': 'masks/1.png', 'fl_x': 1244.28466796875, 'fl_y': 1244.28466796875, 'cx': 640.0, 'cy': 360.0}, 'masked_images/2.png': {'file_path': 'masked_images/2.png

In [116]:
#Write out transforms

transforms = data_1.copy()
transforms["frames"] = list(all_frames.values())
print(transforms)

with open("/dust3r/masked_dust3r/data/jackal_training_data_0/transforms.json", "w") as f:
    json.dump(transforms, f, indent=4)

{'camera_model': 'OPENCV', 'frames': [{'file_path': 'masked_images/0.png', 'transform_matrix': [[0.5832669138908386, -0.6942539215087891, 0.42167627811431885, 0.15659742057323456], [-0.7428847551345825, -0.6658796072006226, -0.06874813884496689, -0.11959019303321838], [0.32851433753967285, -0.27315840125083923, -0.9041363000869751, 0.028364472091197968], [0.0, 0.0, 0.0, 1.0]], 'mask_path': 'masks/0.png', 'fl_x': 1204.841537475586, 'fl_y': 1204.841537475586, 'cx': 640.0, 'cy': 360.0}, {'file_path': 'masked_images/1.png', 'transform_matrix': [[0.5058984756469727, -0.7568375468254089, 0.4138403534889221, 0.1549302637577057], [-0.7972010970115662, -0.5934606194496155, -0.11079257726669312, -0.13495416939258575], [0.3294498920440674, -0.27386417984962463, -0.9035824537277222, 0.02838529460132122], [0.0, 0.0, 0.0, 1.0]], 'mask_path': 'masks/1.png', 'fl_x': 1244.28466796875, 'fl_y': 1244.28466796875, 'cx': 640.0, 'cy': 360.0}, {'file_path': 'masked_images/2.png', 'transform_matrix': [[0.41899